In [2]:
import tensorflow as tf
tf.__version__

'1.15.2'

In [3]:
sess = tf.Session()

In [4]:
import numpy as np

# python control flow vs. Tensorflow graph's

python control flow 와 Tensorflow graph's control flow 는 다르다.

- python control flow 의 구성요소 : `if`, `while`, `for`, etc
- tensorflow control flow 의 구성요소 : `tf.cond`, `tf.while_loop`

# tf.function
- Creates a callable TensorFlow graph from a Python function.
- 파이썬 함수 안의 Tensorflow operator 처리를 진행하는 callable Tensorflow graph 를 만든다.

### alias
- tf.compat.v1.function
- tf.compat.v2.function
- tf.contrib.eager.function

### without eager execution

In [ ]:
def f(x, y):
    return tf.reduce_mean(tf.multiply(x ** 2, 3) + y)

g = tf.function(f) # callable graph

x = tf.constant([[2.0, 3.0]])
y = tf.constant([[3.0, -2.0]])

In [9]:
res = g(x,y)

In [15]:
sess.run(res)

20.0

In [16]:
def f(x, y):
    return tf.reduce_mean(tf.multiply(x ** 2, 3) + y)

g = tf.compat.v1.function(f) # callable graph

x = tf.constant([[2.0, 3.0]])
y = tf.constant([[3.0, -2.0]])

In [19]:
g(x,y)

<tf.Tensor 'PartitionedCall_7:0' shape=() dtype=float32>

In [20]:
sess.run(g(x,y))

20.0

In [24]:
def f(x, y):
    return tf.reduce_mean(tf.multiply(x ** 2, 3) + y)

@tf.function
def h():
    return f(x, y)

g = tf.function(f) # callable graph

x = tf.constant([[2.0, 3.0]])
y = tf.constant([[3.0, -2.0]])

h()

<tf.Tensor 'PartitionedCall_11:0' shape=() dtype=float32>

In [25]:
c = tf.Variable(0)

@tf.function
def f(x):
    c.assign_add(1)
    return x + tf.compat.v1.to_float(c)

assert int(c) == 0
assert f(1.0) == 2.0
assert int(c) == 1
assert f(1.0) == 3.0
assert int(c) == 2

TypeError: int() argument must be a string, a bytes-like object or a number, not 'RefVariable'

In [26]:
tf.executing_eagerly()

False

### with eager execution

In [27]:
import tensorflow.contrib.eager as tfe
tfe.enable_eager_execution()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



ValueError: tf.enable_eager_execution must be called at program startup.

In [3]:
import tensorflow.contrib.eager as tfe
# tfe.enable_eager_execution()
dir(tfe)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__']

In [4]:
import tensorflow as tf
tf.enable_eager_execution()

In [5]:
tf.executing_eagerly()

True

In [6]:
def f(x, y):
    return tf.reduce_mean(tf.multiply(x ** 2, 3) + y)

g = tf.compat.v1.function(f) # callable graph

x = tf.constant([[2.0, 3.0]])
y = tf.constant([[3.0, -2.0]])

g(x,y)

<tf.Tensor: id=16, shape=(), dtype=float32, numpy=20.0>

# tf.py_function

- tensorflow control flow 의 요소들을 이용하지 않고,
- 기존의 python control flow 의 요소들만으로
- tensorflow graph control flow 에 맞는 operation 으로 만들어준다.
- it wraps a Python function func in a once-differentiable TensorFlow operation that executes it with eager execution enabled.
- 특히 이렇게 wrapping 된 operation 은 (with eager execution enabled) input 과 return 에 대해 한번 미분되는 operation 으로 작동한다.

```
tf.py_function(
    func,
    inp,
    Tout,
    name=None
)
```
- func : 제어의 body 이며, 안에 Tensorflow operation 이 쓰일 수 있다.
- inp : input 으로 `tf.Tensor` 나 NumPy Array 이 사용될 수 있다. 물론 `func` 이 갖는 다른 input 들도 명시해야 한다.
- Tout : output 의 타입이다. 보통 `tf.float32` 같은 형태를 기입한다.
- 이렇게 input 으로 들어간 Tensor 들에 대해, return 으로 Tensor 또는 list of Tensors 를 받을 수 있다.
- input 에 대한 return 값들에 대해 once-differential 하여, `tf.gradients` 등을 이용해 미분량을 얻을 수 있다.

In [6]:
def log_huber(x, m):
    if tf.abs(x) <= m:
        return x**2
    else:
        return m**2 * (1 - 2 * tf.math.log(m) + tf.math.log(x**2))

x = tf.compat.v1.placeholder(tf.float32)
m = tf.compat.v1.placeholder(tf.float32)

y = tf.py_function(func=log_huber, inp=[x, m], Tout=tf.float32)
dy_dx = tf.gradients(y, x)[0]

with tf.compat.v1.Session() as sess:
    # The session executes `log_huber` eagerly. Given the feed values below,
    # it will take the first branch, so `y` evaluates to 1.0 and
    # `dy_dx` evaluates to 2.0.
    y, dy_dx = sess.run([y, dy_dx], feed_dict={x: 1.0, m: 2.0})
    print({
        'y': y,
        'dy_dx': dy_dx
    })

{'y': 1.0, 'dy_dx': 2.0}


### debugging 을 위한 사용법

You can also use tf.py_function to debug your models at runtime using Python tools, i.e., you can isolate portions of your code that you want to debug, wrap them in Python functions and insert pdb tracepoints or print statements as desired, and wrap those functions in tf.py_function.

또한 `tf.py_function` 으로 모델의 원하는 부분을 wrapping 하고, pdb 로 tracepoints 잡아서 디버깅할 수도 있다.

In [7]:
def log_huber(x, m):
    if tf.abs(x) <= m:
        return x**2
    else:
        import pdb; pdb.set_trace()
        return m**2 * (1 - 2 * tf.math.log(m) + tf.math.log(x**2))

x = tf.compat.v1.placeholder(tf.float32)
m = tf.compat.v1.placeholder(tf.float32)

y = tf.py_function(func=log_huber, inp=[x, m], Tout=tf.float32)
dy_dx = tf.gradients(y, x)[0]

with tf.compat.v1.Session() as sess:
    # The session executes `log_huber` eagerly. Given the feed values below,
    # it will take the first branch, so `y` evaluates to 1.0 and
    # `dy_dx` evaluates to 2.0.
    y, dy_dx = sess.run([y, dy_dx], feed_dict={x: 4.0, m: 2.0})
    print({
        'y': y,
        'dy_dx': dy_dx
    })

> <ipython-input-7-39a1105a459c>(6)log_huber()
-> return m**2 * (1 - 2 * tf.math.log(m) + tf.math.log(x**2))


(Pdb)  l


  1  	def log_huber(x, m):
  2  	    if tf.abs(x) <= m:
  3  	        return x**2
  4  	    else:
  5  	        import pdb; pdb.set_trace()
  6  ->	        return m**2 * (1 - 2 * tf.math.log(m) + tf.math.log(x**2))
  7  	
  8  	x = tf.compat.v1.placeholder(tf.float32)
  9  	m = tf.compat.v1.placeholder(tf.float32)
 10  	
 11  	y = tf.py_function(func=log_huber, inp=[x, m], Tout=tf.float32)


(Pdb)  m


<tf.Tensor: id=52, shape=(), dtype=float32, numpy=2.0>


...

--Return--
> /opt/conda/lib/python3.7/site-packages/zmq/sugar/socket.py(106)close()->None
-> super(Socket, self).close(linger=linger)


(Pdb)  r


--Return--
> /opt/conda/lib/python3.7/site-packages/zmq/sugar/socket.py(67)__del__()->None
-> self.close()


(Pdb)  r


{'y': 9.545177, 'dy_dx': 2.0}


### `tf.py_function` vs. `tf.py_func`
tf.py_function is similar in spirit to tf.compat.v1.py_func, but unlike the latter, the former lets you use TensorFlow operations in the wrapped Python function. In particular, while tf.compat.v1.py_func only runs on CPUs and wraps functions that take NumPy arrays as inputs and return NumPy arrays as outputs, tf.py_function can be placed on GPUs and wraps functions that take Tensors as inputs, execute TensorFlow operations in their bodies, and return Tensors as outputs.

`tf.py_function`과 `tf.py_func` 은 둘다 graph 에 사용할 operation 을 만들어준다는 틀을 가지고 있지만, 아래와 같은 차이가 있다.
- `tf.py_func` 은 input, return 이 전부 NumPy array 로 제한되며, `tf.py_function` 은 `tf.Tensor` 를 input, return 에 사용할 수 있다.
- `tf.py_func` 은 TensorFlow operations 을 body 에서 사용할 수 없으나, `tf.py_function` 은 사용할 수 있다.
- `tf.py_func` 은 CPU only `tf.py_function` 은 GPU 에서도 돌 수 있다.

다만, `tf.py_function` 도 `tf.py_func` 과 같은 한계를 갖는 것들이 있다.
- `GraphDef` 에 serialized 되어 저장되지 않기 때문에, 다른 환경에서 serialized 된 모델을 사용할 때는 `tf.py_function` 을 사용해서는 안된다.
- `tf.py_function` 을 호출한 파이썬 프로그램의 address space 와 생성된 operation 이 돌아갈 address space 가 같아야 한다. 즉 호출한 프로세스와 생성된 oepration 이 돌아갈 프로세스가 같아야 한다. 이런 사항은 distributed tensorflow 를 사용할 때 특이 유의해야 한다.

### 더 명료한 그래프
without tf.py_function
```python
# Part 1 of the graph
inputs = ...  # in the TF graph

# Get the numpy array and apply func
val = sess.run(inputs)  # get the value of inputs
output_val = func(val)  # numpy array

# Part 2 of the graph
output = tf.placeholder(tf.float32, shape=...)
train_op = ...

# We feed the output_val to the tensor output
sess.run(train_op, feed_dict={output: output_val})
```
with tf.py_function
```python
# Part 1 of the graph
inputs = ...

# call to tf.py_func
output = tf.py_func(func, [inputs], [tf.float32])[0]

# Part 2 of the graph
train_op = ...

# Only one call to sess.run, no need of a intermediate placeholder
sess.run(train_op)
```